# pyrompt Demo

This notebook demonstrates the key features of **pyrompt**: a flexible framework for managing, sharing, and searching prompts and prompt templates.

## Features Covered:
1. Basic prompt and template collections
2. Multi-engine template support (Format, Jinja2, Mustache)
3. Metadata management
4. Template rendering
5. Collection of collections (PromptMall)
6. Integration with oa.prompt_function (if available)
7. Semantic search (if numpy and oa available)
8. GitHub integration (if PyGithub available)

## 1. Basic Setup

In [ ]:
from pyrompt import PromptCollection, TemplateCollection, PromptMall
import tempfile

# Create a temporary directory for this demo
demo_dir = tempfile.mkdtemp()
print(f"Demo directory: {demo_dir}")

## 2. PromptCollection - Managing Simple Prompts

In [ ]:
# Create a prompt collection
prompts = PromptCollection('my_project', base_path=demo_dir)

# Add some system prompts
prompts['python_expert'] = "You are an expert Python programmer with deep knowledge of best practices."
prompts['data_analyst'] = "You are a data analyst specialized in Python, pandas, and visualization."
prompts['code_reviewer'] = "You are an expert code reviewer focusing on correctness and readability."

# List all prompts
print("\nAvailable prompts:")
for key in prompts:
    print(f"  - {key}")

# Access a prompt
print(f"\nPython expert prompt:\n{prompts['python_expert']}")

## 3. TemplateCollection - Templates with Multiple Engines

In [ ]:
# Create a template collection
templates = TemplateCollection('my_project', base_path=demo_dir)

# Add templates with different engines

# 1. Python format strings (default)
templates['greeting.txt'] = "Hello {name}, welcome to {place}!"

# 2. Code review template
templates['code_review.txt'] = """Review this {language} code:

{code}

Focus on: {focus_areas}
"""

# Render templates
print("\n" + "="*50)
print("Format String Templates")
print("="*50)

result = templates.render('greeting.txt', name='Alice', place='Python Land')
print(f"Greeting: {result}")

result = templates.render(
    'code_review.txt',
    language='Python',
    code='def hello(): return "world"',
    focus_areas='type hints and documentation'
)
print(f"\nCode review template:\n{result}")

### 3.1 Jinja2 Templates (if available)

In [ ]:
try:
    import jinja2
    
    # Add Jinja2 template
    templates['advanced.jinja2'] = """Hello {{ name|default('friend') }}!

{% if role %}
You are logged in as: {{ role }}
{% else %}
You are browsing as a guest.
{% endif %}

{% if items %}
Your items:
{% for item in items %}
  - {{ item }}
{% endfor %}
{% endif %}
"""
    
    print("\n" + "="*50)
    print("Jinja2 Template")
    print("="*50)
    
    result = templates.render(
        'advanced.jinja2',
        name='Bob',
        role='admin',
        items=['prompt1', 'prompt2', 'prompt3']
    )
    print(result)
    
    print("\n" + "="*50)
    print("Jinja2 with Defaults")
    print("="*50)
    
    result = templates.render('advanced.jinja2')
    print(result)
    
except ImportError:
    print("Jinja2 not installed. Skipping Jinja2 examples.")
    print("Install with: pip install jinja2")

## 4. Template Parsing - Extract Parameters

In [ ]:
# Parse a template to see what parameters it expects
info = templates.parse('greeting.txt')

print("\nTemplate parsing result:")
print(f"  Placeholders: {info['placeholders']}")
print(f"  Defaults: {info['defaults']}")
print(f"  Metadata: {info['metadata']}")

## 5. Metadata - Add Context to Prompts

In [ ]:
# Create collection with metadata support
prompts_with_meta = PromptCollection(
    'with_metadata',
    base_path=demo_dir,
    with_metadata=True
)

# Add prompt
prompts_with_meta['ml_expert'] = "You are a machine learning expert."

# Add metadata
prompts_with_meta.meta['ml_expert'] = {
    'author': 'thor',
    'version': '1.0',
    'tags': ['AI', 'ML', 'expert'],
    'created': '2025-11-12',
    'description': 'General ML expert system prompt'
}

# Retrieve metadata
meta = prompts_with_meta.meta['ml_expert']
print("\nPrompt metadata:")
for key, value in meta.items():
    print(f"  {key}: {value}")

## 6. PromptMall - Collection of Collections

In [ ]:
# Create a mall to organize multiple collections
mall = PromptMall('my_workspace', base_path=demo_dir)

# Add prompts to different collections
mall['system']['python'] = "You are a Python expert."
mall['system']['javascript'] = "You are a JavaScript expert."

mall['personas']['analyst'] = "You are a data analyst."
mall['personas']['designer'] = "You are a UX designer."

# Access collections
print("\nCollections in mall:")
for collection_name in mall:
    print(f"  {collection_name}:")
    for prompt_key in mall[collection_name]:
        print(f"    - {prompt_key}")

## 7. Integration with oa.prompt_function (Optional)

In [ ]:
try:
    from oa import prompt_function, PromptFuncs
    
    print("\n" + "="*50)
    print("Integration with oa.prompt_function")
    print("="*50)
    
    # Create a template
    templates['explain'] = "Explain {concept} in simple terms."
    
    # Convert to AI function (would require API key to actually call)
    # explain_func = templates.to_prompt_function('explain')
    
    # Create collection of functions
    # funcs = templates.create_prompt_functions()
    
    print("✓ Template can be converted to AI functions")
    print("  Methods available:")
    print("    - to_prompt_function()")
    print("    - to_prompt_json_function()")
    print("    - create_prompt_functions()")
    
except ImportError:
    print("\noa package not installed. Skipping AI function integration.")
    print("Install with: pip install oa")

## 8. Semantic Search (Optional)

In [ ]:
try:
    from pyrompt import SemanticIndex
    import numpy as np
    from oa import embeddings
    
    print("\n" + "="*50)
    print("Semantic Search")
    print("="*50)
    
    # Create a collection with diverse prompts
    search_prompts = PromptCollection('search_demo', base_path=demo_dir)
    search_prompts['python_expert'] = "You are a Python programming expert."
    search_prompts['data_viz'] = "You specialize in data visualization with matplotlib."
    search_prompts['ml_engineer'] = "You are a machine learning engineer."
    search_prompts['web_dev'] = "You are a full-stack web developer."
    
    # Note: This would require OpenAI API access
    # index = SemanticIndex(search_prompts)
    # results = index.search("help with pandas dataframes", top_k=2)
    # for key, score in results:
    #     print(f"  {key}: {score:.3f}")
    
    print("✓ Semantic search available")
    print("  Requires OpenAI API key to use")
    
except ImportError as e:
    print(f"\nSemantic search not available: {e}")
    print("Install with: pip install pyrompt[search]")

## 9. GitHub Integration (Optional)

In [ ]:
try:
    from pyrompt import discover_prompt_collections, GitHubPromptCollection
    
    print("\n" + "="*50)
    print("GitHub Integration")
    print("="*50)
    
    # Discover public collections
    # collections = discover_prompt_collections(search_term='python', min_stars=5)
    # for repo in collections:
    #     print(f"  {repo['name']} ({repo['stars']} stars)")
    
    # Create/access a GitHub collection
    # gh_coll = GitHubPromptCollection(
    #     repo='username/my_prompts_pyrompt',
    #     token='ghp_...',
    #     readonly=False
    # )
    
    print("✓ GitHub integration available")
    print("  Features:")
    print("    - Discover public collections")
    print("    - Fork collections")
    print("    - Clone collections")
    print("    - Sync to GitHub")
    
except ImportError:
    print("\nGitHub integration not available.")
    print("Install with: pip install pyrompt[github]")

## 10. Real-World Example: Code Review Workflow

In [ ]:
# Set up a code review workflow
print("\n" + "="*50)
print("Real-World Example: Code Review Workflow")
print("="*50)

# 1. System prompts
system_prompts = PromptCollection('code_review_system', base_path=demo_dir)
system_prompts['reviewer'] = """You are an expert code reviewer.
Focus on: correctness, efficiency, readability, and best practices."""

# 2. Review templates
review_templates = TemplateCollection('code_review_templates', base_path=demo_dir)

review_templates['python_review.txt'] = """Review this Python code:

```python
{code}
```

Provide:
1. Summary of what the code does
2. Potential issues or bugs
3. Suggestions for improvement
4. Overall rating (1-5 stars)
"""

# 3. Example usage
code_to_review = """
def calculate_average(numbers):
    total = 0
    for num in numbers:
        total = total + num
    return total / len(numbers)
"""

review_prompt = review_templates.render('python_review.txt', code=code_to_review)

print("\nGenerated review prompt:")
print(review_prompt)

print("\n✓ Workflow set up successfully!")
print("  Next step: Use with oa.prompt_function to get AI review")

## Summary

This demo covered:

1. **PromptCollection** - Store and manage simple prompts
2. **TemplateCollection** - Multi-engine template support (Format, Jinja2, Mustache)
3. **Metadata** - Add context and organization to prompts
4. **Template Parsing** - Extract parameters automatically
5. **PromptMall** - Organize multiple collections
6. **oa Integration** - Convert templates to AI functions
7. **Semantic Search** - Find prompts by meaning
8. **GitHub Integration** - Share and discover collections
9. **Real-World Example** - Code review workflow

### Next Steps:

- Install optional dependencies: `pip install pyrompt[all]`
- Set up OpenAI API key for AI functions and semantic search
- Create your own prompt collections
- Share collections on GitHub (name them *_pyrompt)
- Build domain-specific prompt libraries

## Cleanup

In [ ]:
# Clean up the demo directory
import shutil
shutil.rmtree(demo_dir)
print(f"Cleaned up demo directory: {demo_dir}")